In [1]:

%pip install -U langchain-text-splitters langchain-openai langchain-community chromadb pypdf

# Método do langchain para ajudar a fazer a separação dos documentos em chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Embeddings e LLM
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
# Banco vetorial
from langchain_community.vectorstores import Chroma
# Carregar arquivo PDF
from langchain_community.document_loaders import PyPDFLoader
# Prompt para juntar a pergunta relevante com o Chunk
from langchain.chains.question_answering import load_qa_chain



  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [certifi]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key


In [4]:
# Load dos modelos (Embeddings e LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=200)


In [5]:
# Carregar o PDF
pdf_link = "EntendendoAlgoritmos.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
# Vai iniciar e carregar o PDF em páginas
pages = loader.load_and_split()


In [6]:
# Separar em Chunks (Pedaços de documentos)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [7]:
# Salvar no Vector DB - Chroma. Ao fazer isso uma pasta será criada com nome "text_index"
# Pode demorar um pouco
db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()


/tmp/ipykernel_15111/2130377404.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [8]:
# Carregar os arquivos do DB
vector_db = Chroma(persist_directory="text_index", embedding_function=embeddings_model)
# Método para recuperar do vector_db os chunks relevantes (nesse caso os 3 mais relevantes)
retriever = vector_db.as_retriever(search_kwargs ={"k":3})
# Construção da cadeia de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

/tmp/ipykernel_15111/1264652388.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory="text_index", embedding_function=embeddings_model)
/tmp/ipykernel_15111/1264652388.py:6: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides o

In [9]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
    return answer

In [12]:
user_question = input("User: ")
answer = ask(user_question)
print("Answer: ", answer)

Answer:  Sim, o nome do segundo capítulo deste livro é "Você aprenderá duas estruturas de dados fundamentais: arrays e listas encadeadas".
